In [2]:
pip install pandas numpy yfinance scikit-learn matplotlib seaborn plotly flask python-dotenv tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
import pandas as pd

# # IT Companies
# IT_companies = [
#     'TCS.NS',
#     'INFY.NS',
#     'LTIM.NS',
#     'TECHM.NS',
#     'WIPRO.NS',
#     'ACN',
#     'CTSH',
#     'BSOFT.NS'
# ]

# # Non-IT Companies
# NonIT_companies = [
#     'HDFCBANK.NS',
#     'G',
#     'LT.NS',
#     'RELIANCE.NS',
#     'TATASTEEL.NS'   # Replaced TATAMOTORS (more stable)
# ]

COMPANIES = {
    # -------- IT --------
    "TCS": "TCS.NS",
    "Infosys": "INFY.NS",
    "LTIMindtree": "LTIM.NS",
    "Tech Mahindra": "TECHM.NS",
    "Wipro": "WIPRO.NS",
    "HCL Tech": "HCLTECH.NS",
    "Accenture": "ACN",
    "Cognizant": "CTSH",
    "Birlasoft": "BSOFT.NS",

    # -------- Non-IT --------
    "Sagility": "SAGILITY.NS",
    "Genpact": "G",
    "L&T": "LT.NS",
    "Reliance": "RELIANCE.NS",
    "Tata Consumer": "TATACONSUM.NS"
}


all_stocks = list(COMPANIES.values())

# Download data safely
all_data = {}

for ticker in all_stocks:
    print(f"Downloading {ticker}...")
    try:
        df = yf.download(ticker, period="5y", 
            auto_adjust=True,   # important
            threads=False,  # Better than manual date
            progress=False
        )

        if not df.empty:
            all_data[ticker] = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        else:
            print(f"{ticker} returned empty data.")

    except Exception as e:
        print(f"Error downloading {ticker}: {e}")

# Combine
data = pd.concat(all_data, axis=1)

# Flatten columns
data.columns = [f"{col[0]}_{col[1]}" for col in data.columns]

# Save CSV
output_folder = r'C:\Users\Sai\OneDrive\Documents\uma_project'
os.makedirs(output_folder, exist_ok=True)

file_path = os.path.join(output_folder, 'stock_price_5yrs.csv')
data.to_csv(file_path)

print("\nData saved successfully!")
print("Start Date:", data.index.min())
print("End Date:", data.index.max())



1 Failed download:
['TCS.NS']: TypeError("'NoneType' object is not subscriptable")


TCS.NS returned empty data.



1 Failed download:
['INFY.NS']: TypeError("'NoneType' object is not subscriptable")


INFY.NS returned empty data.

Data saved successfully!
Start Date: 2021-02-18 00:00:00
End Date: 2026-02-18 00:00:00


In [ ]:
# Loading the dataset
import pandas as pd

pd.options.display.max_rows = 100
df = pd.read_csv("stock_price_5yrs.csv")
print(df.to_string())

            Date  LTIM.NS_Open  LTIM.NS_High  LTIM.NS_Low  LTIM.NS_Close  LTIM.NS_Volume  TECHM.NS_Open  TECHM.NS_High  TECHM.NS_Low  TECHM.NS_Close  TECHM.NS_Volume  WIPRO.NS_Open  WIPRO.NS_High  WIPRO.NS_Low  WIPRO.NS_Close  WIPRO.NS_Volume  HCLTECH.NS_Open  HCLTECH.NS_High  HCLTECH.NS_Low  HCLTECH.NS_Close  HCLTECH.NS_Volume    ACN_Open    ACN_High     ACN_Low   ACN_Close  ACN_Volume  CTSH_Open  CTSH_High   CTSH_Low  CTSH_Close  CTSH_Volume  BSOFT.NS_Open  BSOFT.NS_High  BSOFT.NS_Low  BSOFT.NS_Close  BSOFT.NS_Volume  SAGILITY.NS_Open  SAGILITY.NS_High  SAGILITY.NS_Low  SAGILITY.NS_Close  SAGILITY.NS_Volume     G_Open     G_High      G_Low    G_Close    G_Volume   LT.NS_Open   LT.NS_High    LT.NS_Low  LT.NS_Close  LT.NS_Volume  RELIANCE.NS_Open  RELIANCE.NS_High  RELIANCE.NS_Low  RELIANCE.NS_Close  RELIANCE.NS_Volume  TATACONSUM.NS_Open  TATACONSUM.NS_High  TATACONSUM.NS_Low  TATACONSUM.NS_Close  TATACONSUM.NS_Volume
0     2021-02-18   3618.364039   3748.079471  3614.924531    3648.1

In [5]:
df.shape

(1298, 61)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  1298 non-null   object 
 1   LTIM.NS_Open          1237 non-null   float64
 2   LTIM.NS_High          1237 non-null   float64
 3   LTIM.NS_Low           1237 non-null   float64
 4   LTIM.NS_Close         1237 non-null   float64
 5   LTIM.NS_Volume        1237 non-null   float64
 6   TECHM.NS_Open         1237 non-null   float64
 7   TECHM.NS_High         1237 non-null   float64
 8   TECHM.NS_Low          1237 non-null   float64
 9   TECHM.NS_Close        1237 non-null   float64
 10  TECHM.NS_Volume       1237 non-null   float64
 11  WIPRO.NS_Open         1237 non-null   float64
 12  WIPRO.NS_High         1237 non-null   float64
 13  WIPRO.NS_Low          1237 non-null   float64
 14  WIPRO.NS_Close        1237 non-null   float64
 15  WIPRO.NS_Volume      

In [7]:
df.describe()

,LTIM.NS_Open,LTIM.NS_High,LTIM.NS_Low,LTIM.NS_Close,LTIM.NS_Volume,TECHM.NS_Open,TECHM.NS_High,TECHM.NS_Low,TECHM.NS_Close,TECHM.NS_Volume,...,RELIANCE.NS_Open,RELIANCE.NS_High,RELIANCE.NS_Low,RELIANCE.NS_Close,RELIANCE.NS_Volume,TATACONSUM.NS_Open,TATACONSUM.NS_High,TATACONSUM.NS_Low,TATACONSUM.NS_Close,TATACONSUM.NS_Volume
count,1237.000000,1237.000000,1237.000000,1237.000000,1.237000e+03,1237.000000,1237.000000,1237.000000,1237.000000,1.237000e+03,...,1237.000000,1237.000000,1237.000000,1237.000000,1.237000e+03,1237.000000,1237.000000,1237.000000,1237.000000,1.237000e+03
mean,5070.384664,5131.916594,4999.960607,5064.683629,4.045149e+05,1235.932483,1250.189200,1220.675180,1235.510593,2.567278e+06,...,1236.126983,1247.218524,1224.686108,1235.571491,1.346273e+07,902.985766,912.499154,892.700073,902.714783,1.781788e+06
std,805.020443,812.363003,795.065534,803.522124,3.632120e+05,263.920287,266.665079,260.987179,263.772397,1.762751e+06,...,174.777025,175.514758,174.177134,174.816107,7.380660e+06,184.689888,186.632183,182.833752,184.993406,1.465985e+06
min,3383.058055,3392.481587,3321.804887,3377.073975,0.000000e+00,785.165385,796.412592,768.000781,771.232239,0.000000e+00,...,858.005344,868.071522,850.954480,862.040833,0.000000e+00,567.605548,584.585636,555.147075,572.319641,0.000000e+00
25%,4495.035168,4548.477444,4441.358127,4484.098145,2.295100e+05,984.663597,993.991492,974.953520,984.116516,1.534262e+06,...,1112.117039,1124.716394,1101.584185,1111.158447,8.918320e+06,743.078502,750.940323,736.799261,742.553223,9.617350e+05
50%,5051.187709,5103.059795,5000.338472,5049.046875,3.166410e+05,1219.963004,1235.884782,1201.663406,1219.017090,2.188506e+06,...,1207.065942,1217.157279,1196.643063,1206.987671,1.180952e+07,848.961542,854.728739,840.532534,847.975708,1.391781e+06
75%,5682.076163,5741.000000,5588.805876,5663.943848,4.657230e+05,1465.000000,1484.061137,1447.907290,1462.586060,3.018243e+06,...,1401.004550,1414.351234,1387.232416,1399.908936,1.604373e+07,1099.898280,1108.199951,1085.900024,1099.267944,2.105780e+06
max,7190.691513,7227.644814,7082.212167,7201.882324,5.261322e+06,1845.000000,1854.000000,1739.900024,1768.400024,2.043193e+07,...,1593.000000,1611.800049,1578.199951,1592.300049,9.146198e+07,1245.123857,1251.038935,1205.300049,1243.694458,1.989524e+07


In [8]:
df.isna().sum()

Date                      0
LTIM.NS_Open             61
LTIM.NS_High             61
LTIM.NS_Low              61
LTIM.NS_Close            61
LTIM.NS_Volume           61
TECHM.NS_Open            61
TECHM.NS_High            61
TECHM.NS_Low             61
TECHM.NS_Close           61
TECHM.NS_Volume          61
WIPRO.NS_Open            61
WIPRO.NS_High            61
WIPRO.NS_Low             61
WIPRO.NS_Close           61
WIPRO.NS_Volume          61
HCLTECH.NS_Open          61
HCLTECH.NS_High          61
HCLTECH.NS_Low           61
HCLTECH.NS_Close         61
HCLTECH.NS_Volume        61
ACN_Open                 43
ACN_High                 43
ACN_Low                  43
ACN_Close                43
ACN_Volume               43
CTSH_Open                43
CTSH_High                43
CTSH_Low                 43
CTSH_Close               43
CTSH_Volume              43
BSOFT.NS_Open            61
BSOFT.NS_High            61
BSOFT.NS_Low             61
BSOFT.NS_Close           61
BSOFT.NS_Volume     

<h3>code for handling null values, new features, create targets/features<h3>

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("stock_price_5yrs.csv", parse_dates=['Date'])
df = df.sort_values('Date').set_index('Date')

# Separate columns
price_cols = [c for c in df.columns if any(x in c for x in ['Open','High','Low','Close']) and 'Volume' not in c]
volume_cols = [c for c in df.columns if 'Volume' in c]

# 1️⃣ Handle nulls
df[price_cols] = df[price_cols].ffill().bfill()
df[volume_cols] = df[volume_cols].ffill().bfill().fillna(0)

# 2️⃣ Create features in separate DataFrames to avoid fragmentation
feature_dfs = []

for col in price_cols:
    temp = pd.DataFrame({
        f'{col}_MA20': df[col].rolling(20).mean(),
        f'{col}_MA50': df[col].rolling(50).mean(),
        f'{col}_Return': df[col].pct_change() * 100,
        f'{col}_Volatility': df[col].rolling(20).std(),
        f'{col}_Next': df[col].shift(-1) if 'Close' in col else None  # Next-day Close only for Close columns
    }, index=df.index)
    
    # Remove None columns
    temp = temp[[c for c in temp.columns if temp[c].notna().any()]]
    
    feature_dfs.append(temp)

# Concatenate all features with original df
df = pd.concat([df, *feature_dfs], axis=1)

# 3️⃣ Drop rows with NaNs created by rolling / shift
df = df.dropna()

# 4️⃣ Save final dataset
df.to_csv("stock_features_ready.csv")

print("Dataset ready! Columns now include features and targets.")


Dataset ready! Columns now include features and targets.


In [10]:
df.shape

(1248, 264)

In [11]:
# All feature columns (exclude Volume if you like)
feature_cols = [c for c in df.columns if any(x in c for x in ['MA20','MA50','Return','Volatility'])]
target_cols = [c for c in df.columns if '_Next' in c]

X = df[feature_cols]
y = df[target_cols]


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

timesteps = 60

results = []

company_models = {}
scalers_X = {}
scalers_y = {}
company_features_dict = {}

for target in target_cols:

    print(f"\nTraining model for: {target}")

    # Select features only related to this company
    company_prefix = target.replace("_Close_Next", "")
    company_features = [
        c for c in feature_cols 
        if company_prefix in c and c != target
    ]
    
    # Save features for prediction
    company_features_dict[target] = company_features

    X = df[company_features]
    y = df[[target]]

    # --- Create Sequences ---
    X_seq, y_seq = [], []

    for i in range(timesteps, len(df)):
        X_seq.append(X.iloc[i-timesteps:i].values)
        y_seq.append(y.iloc[i].values)

    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)

    # --- Scaling (separate per company) ---
    # --- Train/Test Split (BEFORE scaling) ---
    train_size = int(len(X_seq) * 0.8)

    X_train = X_seq[:train_size]
    X_test = X_seq[train_size:]

    y_train = y_seq[:train_size]
    y_test = y_seq[train_size:]

    # --- Scaling (fit ONLY on train) ---
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    nsamples, ntimesteps, nfeatures = X_train.shape

    # Flatten only TRAIN
    X_train_flat = X_train.reshape((nsamples * ntimesteps, nfeatures))
    scaler_X.fit(X_train_flat)

    # Transform train
    X_train_scaled = scaler_X.transform(X_train_flat).reshape(X_train.shape)

    # Transform test (do NOT fit again)
    X_test_flat = X_test.reshape((X_test.shape[0] * ntimesteps, nfeatures))
    X_test_scaled = scaler_X.transform(X_test_flat).reshape(X_test.shape)

    # Scale y
    scaler_y.fit(y_train.reshape(-1,1))

    y_train_scaled = scaler_y.transform(y_train.reshape(-1,1))
    y_test_scaled = scaler_y.transform(y_test.reshape(-1,1))


    # --- Model ---
    model = Sequential()
    model.add(LSTM(126, return_sequences=True, input_shape=(timesteps, nfeatures)))
    model.add(Dropout(0.2))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')

    early_stop = EarlyStopping(monitor='val_loss',
                                patience=10,
                                restore_best_weights=True)

    model.fit(
        X_train_scaled, 
        y_train_scaled,
        epochs=100,
        batch_size=16,
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=0
    )

    # --- Evaluation ---
    y_pred_scaled = model.predict(X_test_scaled, verbose=0)

    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_actual = y_test

    rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
    mae = mean_absolute_error(y_actual, y_pred)
    mape = np.mean(np.abs((y_actual - y_pred) / (y_actual + 1e-8))) * 100

    from sklearn.linear_model import LinearRegression

    # Flatten for Linear Regression
    X_flat_train = X_train_scaled.reshape(X_train_scaled.shape[0], -1)
    X_flat_test = X_test_scaled.reshape(X_test_scaled.shape[0], -1)

    lr_model = LinearRegression()
    lr_model.fit(X_flat_train, y_train)

    lr_pred = lr_model.predict(X_flat_test)

    lr_rmse = np.sqrt(mean_squared_error(y_actual, lr_pred))
    lr_mae = mean_absolute_error(y_actual, lr_pred)
    lr_mape = np.mean(np.abs((y_actual - lr_pred) / (y_actual + 1e-8))) * 100

    results.append([
        target,
        rmse, mae, mape,
        lr_rmse, lr_mae, lr_mape
    ])

    # Save model and scalers for this company
    company_models[target] = model
    scalers_X[target] = scaler_X
    scalers_y[target] = scaler_y




Training model for: LTIM.NS_Close_Next


c:\Users\Sai\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Training model for: TECHM.NS_Close_Next


c:\Users\Sai\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Training model for: WIPRO.NS_Close_Next


c:\Users\Sai\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# --- 5. Evaluate Model ---
evaluation_df = pd.DataFrame(results,
                             columns=[
                                 "Company",
                                 "LSTM_RMSE",
                                 "LSTM_MAE",
                                 "LSTM_MAPE(%)",
                                 "LR_RMSE",
                                 "LR_MAE",
                                 "LR_MAPE(%)"
                             ])

print("\nFinal Model Comparison:")
print(evaluation_df)



Final Model Comparison:
                 Company   LSTM_RMSE    LSTM_MAE  LSTM_MAPE(%)     LR_RMSE  \
0      TCS.NS_Close_Next  100.192199   71.331192      2.326065  265.405719   
1     INFY.NS_Close_Next   53.688349   41.169492      2.752919  177.614864   
2     LTIM.NS_Close_Next  160.616845  121.727116      2.315014  645.254883   
3    TECHM.NS_Close_Next   61.659269   46.520541      3.122716  185.451397   
4    WIPRO.NS_Close_Next    7.538515    5.619681      2.343049   25.165553   
5  HCLTECH.NS_Close_Next   94.944242   73.665095      4.948966  172.962448   
6         ACN_Close_Next   13.473979   11.213834      4.285045   40.035979   
7        CTSH_Close_Next    3.067638    2.399452      3.225372    9.115186   
8    BSOFT.NS_Close_Next   19.891928   15.837148      3.939571   60.259795   
9           G_Close_Next    1.948513    1.503032      3.482600    6.328030   

       LR_MAE  LR_MAPE(%)  
0  211.500846    6.694696  
1  131.672487    8.649502  
2  490.671225    9.389961  
3  1

In [ ]:
import plotly.graph_objects as go
import pandas as pd

companies = [c.replace('_Next','') for c in target_cols]

for target in target_cols:
    comp = target.replace('_Next','')
    print(f"\nPredicting 7-day future for: {comp}")

    model = company_models[target]
    scaler_X = scalers_X[target]
    scaler_y = scalers_y[target]

    # Features for this company
    company_features = company_features_dict[target]
    X = df[company_features].values  # full feature data

    # Last 60 days
    last_seq = X[-timesteps:].copy()
    last_seq_scaled = scaler_X.transform(last_seq).reshape(1, timesteps, len(company_features))

    future_preds = []
    for _ in range(7):
        next_pred_scaled = model.predict(last_seq_scaled, verbose=0)
        next_pred = scaler_y.inverse_transform(next_pred_scaled)[0, 0]
        future_preds.append(next_pred)

        # Update last_seq_scaled
        new_features = last_seq_scaled[0, -1, :].copy()
        new_features[-1] = next_pred_scaled.item()
        last_seq_scaled = np.roll(last_seq_scaled, -1, axis=1)
        last_seq_scaled[0, -1, :] = new_features

    # Dates
    actual_prices = df[comp].values[-30:]
    last_date = df.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7)

    # Plotly figure
    fig = go.Figure()

    # Actual Prices
    fig.add_trace(go.Scatter(
        x=df.index[-30:], 
        y=actual_prices,
        mode='lines+markers',
        name='Actual Price',
        fill='tozeroy',
        hovertemplate='<b>Actual</b><br>Date: %{x}<br>Price: ₹%{y:,.2f}<extra></extra>'  # Custom tooltip

    ))

    # Predicted Prices
    fig.add_trace(go.Scatter(
        x=future_dates,
        y=future_preds,
        mode='lines+markers',
        name='Predicted Price',
        line=dict(dash='dash', color='red'),
        hovertemplate='<b>Predicted</b><br>Date: %{x}<br>Price: ₹%{y:,.2f}<extra></extra>'  # Custom tooltip

    ))

    fig.update_layout(
        title=f"{comp} Close Price: Actual vs Predicted (7-day forecast)",
        xaxis_title='Date',
        yaxis_title='Price',
        legend=dict(x=1,      # Right edge (0=left, 1=right)
                    y=1,      # Top edge (0=bottom, 1=top)
                    # xanchor='right',  # Anchor legend's right side to x position
                    # yanchor='top' # Anchor legend's top to y position
                ),
        template='plotly_white',
        width=500,   # set width in pixels
        height=300 ,
        hovermode='x unified'
    )
    fig.show()



Predicting 7-day future for: TCS.NS_Close



Predicting 7-day future for: INFY.NS_Close



Predicting 7-day future for: LTIM.NS_Close



Predicting 7-day future for: TECHM.NS_Close



Predicting 7-day future for: WIPRO.NS_Close



Predicting 7-day future for: HCLTECH.NS_Close



Predicting 7-day future for: ACN_Close



Predicting 7-day future for: CTSH_Close



Predicting 7-day future for: BSOFT.NS_Close



Predicting 7-day future for: G_Close


In [ ]:
import os
import joblib

os.makedirs("models", exist_ok=True)

# Save each model separately
for name, model in company_models.items():
    model.save(f"models/{name}.keras")

# Save scalers & feature dict
joblib.dump(scalers_X, "scalers_X.pkl")
joblib.dump(scalers_y, "scalers_y.pkl")
joblib.dump(company_features_dict, "features.pkl")

print("Models and scalers saved properly.")


Models and scalers saved properly.
